In [1]:
import pandas as pd
import numpy as np
from random import sample
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
import glob
import cv2

from google.colab import drive

import matplotlib.pyplot as plt
import keras as ks
from keras import layers
from keras.models import Sequential
import tensorflow as tf
from keras.backend import categorical_crossentropy

from sklearn.metrics import accuracy_score

import os

In [2]:
#drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
image_size = (120, 120)
batch_size = 32
## /content/drive/MyDrive/BT5153 Applied ML Project /5 Classes 5k each for train (cropped)
train_directory = '/content/drive/MyDrive/Colab Notebooks/BT5153 Applied ML Project /5 Classes 5k each for train (cropped)/Training_cropped'
val_directory = '/content/drive/MyDrive/Colab Notebooks/BT5153 Applied ML Project /5 Classes 5k each for train (cropped)/Valid_cropped'
test_directory = '/content/drive/MyDrive/Colab Notebooks/BT5153 Applied ML Project /5 Classes 5k each for train (cropped)/Test_cropped'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(val_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_directory, label_mode='categorical', image_size=image_size, batch_size=batch_size)

train_ds = train_ds.prefetch(buffer_size=batch_size)
val_ds = val_ds.prefetch(buffer_size=batch_size)
test_ds = test_ds.prefetch(buffer_size=batch_size)

Found 25000 files belonging to 5 classes.
Found 27427 files belonging to 5 classes.
Found 34281 files belonging to 5 classes.


In [4]:
def make_model(input_shape, num_classes):
    inputs = ks.Input(shape=input_shape)
    x = layers.Rescaling(1.0/255)(inputs)
    x = layers.Conv2D(32, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(64, 3, strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    previous_block_activation = x
    
    for size in (128, 256, 512, 728):
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)      
        
        x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
        
        residual = layers.Conv2D(size, 1, strides=2, padding='same')(previous_block_activation)
        x = layers.add([x, residual])
        previous_block_activation = x

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(1024, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)
        
    return ks.Model(inputs, outputs)

In [7]:
model = make_model(image_size+(3,), 5)

epochs = 1000
saved_model = '/content/drive/MyDrive/Colab Notebooks/5153 weights/CNN_5C5K_(cropped)_{epoch}.h5'
callbacks = [ks.callbacks.ModelCheckpoint(saved_model,verbose=1, save_best_only=True), 
             ks.callbacks.EarlyStopping(patience=20,verbose=1, restore_best_weights=True)]

model.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds)

Epoch 1/1000
782/782 [==============================] - ETA: 0s - loss: 1.0893 - accuracy: 0.5701
Epoch 1: val_loss improved from inf to 1.68925, saving model to /content/drive/MyDrive/Colab Notebooks/5153 weights/CNN_5C5K_(cropped)_1.h5
782/782 [==============================] - 6636s 8s/step - loss: 1.0893 - accuracy: 0.5701 - val_loss: 1.6893 - val_accuracy: 0.4713
Epoch 2/1000
781/782 [============================>.] - ETA: 0s - loss: 0.8348 - accuracy: 0.6898
Epoch 2: val_loss did not improve from 1.68925
782/782 [==============================] - 46s 58ms/step - loss: 0.8347 - accuracy: 0.6898 - val_loss: 1.8254 - val_accuracy: 0.4684
Epoch 3/1000
781/782 [============================>.] - ETA: 0s - loss: 0.7033 - accuracy: 0.7426
Epoch 3: val_loss improved from 1.68925 to 1.44004, saving model to /content/drive/MyDrive/Colab Notebooks/5153 weights/CNN_5C5K_(cropped)_3.h5
782/782 [==============================] - 48s 62ms/step - loss: 0.7036 - accuracy: 0.7426 - val_loss: 1.4400

In [9]:
true = []
pred = []
batches = int(np.ceil(34281/32))
i = 1
for image, label in test_ds:
  pred = np.concatenate([pred, 
                         np.argmax(model.predict(image), axis=-1)])
  true = np.concatenate([true,
                         np.argmax(label.numpy(), axis=-1)])
  print('{}/{}'.format(i, batches))
  i += 1

1/1072
2/1072
3/1072
4/1072
5/1072
6/1072
7/1072
8/1072
9/1072
10/1072
11/1072
12/1072
13/1072
14/1072
15/1072
16/1072
17/1072
18/1072
19/1072
20/1072
21/1072
22/1072
23/1072
24/1072
25/1072
26/1072
27/1072
28/1072
29/1072
30/1072
31/1072
32/1072
33/1072
34/1072
35/1072
36/1072
37/1072
38/1072
39/1072
40/1072
41/1072
42/1072
43/1072
44/1072
45/1072
46/1072
47/1072
48/1072
49/1072
50/1072
51/1072
52/1072
53/1072
54/1072
55/1072
56/1072
57/1072
58/1072
59/1072
60/1072
61/1072
62/1072
63/1072
64/1072
65/1072
66/1072
67/1072
68/1072
69/1072
70/1072
71/1072
72/1072
73/1072
74/1072
75/1072
76/1072
77/1072
78/1072
79/1072
80/1072
81/1072
82/1072
83/1072
84/1072
85/1072
86/1072
87/1072
88/1072
89/1072
90/1072
91/1072
92/1072
93/1072
94/1072
95/1072
96/1072
97/1072
98/1072
99/1072
100/1072
101/1072
102/1072
103/1072
104/1072
105/1072
106/1072
107/1072
108/1072
109/1072
110/1072
111/1072
112/1072
113/1072
114/1072
115/1072
116/1072
117/1072
118/1072
119/1072
120/1072
121/1072
122/1072
123/1072
1

In [10]:
accuracy_score(true, pred)

0.6354248709197514

In [11]:
model_results = pd.DataFrame({'true':true,
                              'pred':pred})

model_results.to_csv('/content/drive/MyDrive/Colab Notebooks/5153 weights/CNN_cropped_test_preds.csv')